In [1]:
#!/usr/bin/env python

"""
This application presents a 'console' prompt to the user asking for read commands
which create ReadPropertyRequest PDUs, then lines up the coorresponding ReadPropertyACK
and prints the value.
"""

import sys

from bacpypes.debugging import bacpypes_debugging, ModuleLogger
from bacpypes.consolelogging import ConfigArgumentParser
from bacpypes.consolecmd import ConsoleCmd

from bacpypes.core import run, enable_sleeping
from bacpypes.iocb import IOCB

from bacpypes.pdu import Address
from bacpypes.apdu import ReadPropertyRequest, ReadPropertyACK
from bacpypes.primitivedata import Unsigned
from bacpypes.constructeddata import Array

from bacpypes.app import BIPSimpleApplication
from bacpypes.object import get_object_class, get_datatype
from bacpypes.service.device import LocalDeviceObject

# some debugging
_debug = 0
_log = ModuleLogger(globals())

# globals
this_application = None

In [2]:
#
#   ReadPropertyConsoleCmd
#

@bacpypes_debugging
class ReadPropertyConsoleCmd(ConsoleCmd):

    def do_read(self, args):
        """read <addr> <type> <inst> <prop> [ <indx> ]"""
        args = args.split()
        if _debug: ReadPropertyConsoleCmd._debug("do_read %r", args)

        try:
            addr, obj_type, obj_inst, prop_id = args[:4]

            if obj_type.isdigit():
                obj_type = int(obj_type)
            elif not get_object_class(obj_type):
                raise ValueError("unknown object type")
            print( obj_type)
            print(get_object_class(obj_type))
            print(get_object_class(obj_type)._properties)
            
            obj_inst = int(obj_inst)

            datatype = get_datatype(obj_type, prop_id)
            if not datatype:
                raise ValueError("invalid property for object type")

            # build a request
            request = ReadPropertyRequest(
                objectIdentifier=(obj_type, obj_inst),
                propertyIdentifier=prop_id,
                )
            request.pduDestination = Address(addr)

            if len(args) == 5:
                request.propertyArrayIndex = int(args[4])
            if _debug: ReadPropertyConsoleCmd._debug("    - request: %r", request)

            # make an IOCB
            iocb = IOCB(request)
            if _debug: ReadPropertyConsoleCmd._debug("    - iocb: %r", iocb)

            # give it to the application
            this_application.request_io(iocb)

            # wait for it to complete
            iocb.wait()

            # do something for error/reject/abort
            if iocb.ioError:
                sys.stdout.write(str(iocb.ioError) + '\n')

            # do something for success
            elif iocb.ioResponse:
                apdu = iocb.ioResponse

                # should be an ack
                if not isinstance(apdu, ReadPropertyACK):
                    if _debug: ReadPropertyConsoleCmd._debug("    - not an ack")
                    return

                # find the datatype
                datatype = get_datatype(apdu.objectIdentifier[0], apdu.propertyIdentifier)
                if _debug: ReadPropertyConsoleCmd._debug("    - datatype: %r", datatype)
                if not datatype:
                    raise TypeError("unknown datatype")

                # special case for array parts, others are managed by cast_out
                if issubclass(datatype, Array) and (apdu.propertyArrayIndex is not None):
                    if apdu.propertyArrayIndex == 0:
                        value = apdu.propertyValue.cast_out(Unsigned)
                    else:
                        value = apdu.propertyValue.cast_out(datatype.subtype)
                else:
                    value = apdu.propertyValue.cast_out(datatype)
                if _debug: ReadPropertyConsoleCmd._debug("    - value: %r", value)

                sys.stdout.write(str(value) + '\n')
                if hasattr(value, 'debug_contents'):
                    value.debug_contents(file=sys.stdout)
                sys.stdout.flush()

            # do something with nothing?
            else:
                if _debug: ReadPropertyConsoleCmd._debug("    - ioError or ioResponse expected")

        except Exception as error:
            ReadPropertyConsoleCmd._exception("exception: %r", error)

    def do_rtn(self, args):
        """rtn <addr> <net> ... """
        args = args.split()
        if _debug: ReadPropertyConsoleCmd._debug("do_rtn %r", args)

        # safe to assume only one adapter
        adapter = this_application.nsap.adapters[0]
        if _debug: ReadPropertyConsoleCmd._debug("    - adapter: %r", adapter)

        # provide the address and a list of network numbers
        router_address = Address(args[0])
        network_list = [int(arg) for arg in args[1:]]

        # pass along to the service access point
        this_application.nsap.add_router_references(adapter, router_address, network_list)

In [ ]:
def DiscoverIP():
    try:
        import netifaces as ni
    except ImportError:
        print('Please install Modules "netifaces"')
        raise
    try:
        import ipaddress as ip
    except ImportError:
        print('Please install Module "ipaddress"')
        raise
    
    for x in ni.interfaces(): #for each interface on the system
        try:
            addrs=ni.ifaddresses(x) # obtain all addresses
            ipaddr = addrs[ni.AF_INET][0]['addr']  # IPv4 addressing (use AF_INET6 for IPv6)
            ipmask = addrs[ni.AF_INET][0]['netmask']  # IPv4 addressing (use AF_INET6 for IPv6)
            if ipaddr.startswith( '127.'):
                continue
            return ipaddr+'/'+ str(ip.IPv4Network(ipaddr+'/'+ipmask, strict=False).prefixlen)
        except:
            pass # skip all errors as some devices have no AF_INET


In [ ]:
#
#   __main__
#

def main():
    global this_application

    # parse the command line arguments
    args = ConfigArgumentParser(description=__doc__).parse_args()

    if _debug: _log.debug("initialization")
    if _debug: _log.debug("    - args: %r", args)

    # make a device object
    this_device = LocalDeviceObject(
        objectName=args.ini.objectname,
        objectIdentifier=int(args.ini.objectidentifier),
        maxApduLengthAccepted=int(args.ini.maxapdulengthaccepted),
        segmentationSupported=args.ini.segmentationsupported,
        vendorIdentifier=int(args.ini.vendoridentifier),
        )

    # make a simple application
    this_application = BIPSimpleApplication(this_device, args.ini.address)

    # get the services supported
    services_supported = this_application.get_services_supported()
    if _debug: _log.debug("    - services_supported: %r", services_supported)

    # let the device object know
    this_device.protocolServicesSupported = services_supported.value

    # make a console
    this_console = ReadPropertyConsoleCmd()
    if _debug: _log.debug("    - this_console: %r", this_console)

    # enable sleeping will help with threads
    enable_sleeping()

    _log.debug("running")

    run()

    _log.debug("fini")

if __name__ == "__main__":
    import sys
    sys.argv = ['xxx']
    main()


read 192.168.11.57 analogInput 0 85


ERROR:__main__.ReadPropertyConsoleCmd:exception: ValueError('invalid property for object type',)
Traceback (most recent call last):
  File "<ipython-input-2-c96b0cf66e7a>", line 28, in do_read
    raise ValueError("invalid property for object type")
ValueError: invalid property for object type


analogInput
<class 'bacpypes.object.AnalogInputObject'>
{'timeDelayNormal': <bacpypes.object.OptionalProperty object at 0x70edd0d0>, 'maxPresValue': <bacpypes.object.OptionalProperty object at 0x70ed9e70>, 'deviceType': <bacpypes.object.OptionalProperty object at 0x70ed9d70>, 'eventMessageTexts': <bacpypes.object.OptionalProperty object at 0x70edd030>, 'description': <bacpypes.object.ReadableProperty object at 0x70ec16f0>, 'outOfService': <bacpypes.object.ReadableProperty object at 0x70ed9df0>, 'eventAlgorithmInhibit': <bacpypes.object.OptionalProperty object at 0x70edd0b0>, 'notifyType': <bacpypes.object.OptionalProperty object at 0x70ed9fd0>, 'limitEnable': <bacpypes.object.OptionalProperty object at 0x70ed9f70>, 'eventState': <bacpypes.object.ReadableProperty object at 0x70ed9db0>, 'covIncrement': <bacpypes.object.OptionalProperty object at 0x70ed9eb0>, 'propertyList': <bacpypes.object.ReadableProperty object at 0x70ec1810>, 'minPresValue': <bacpypes.object.OptionalProperty object a

noResponse
